In [1]:
from collections import OrderedDict
import glob
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import open3d as o3d
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

from learning3d.models import PointNet, Classifier
from learning3d.data_utils import ClassificationData, ModelNet40Data

# Data Loading

In [4]:
testset = ClassificationData(ModelNet40Data(train=True, num_points=1024, download=True))
test_loader = DataLoader(testset, batch_size=32, shuffle=False, drop_last=False)

In [5]:
modelnet_labels = [
    'airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car',
    'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot',
    'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor',
    'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink',
    'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase',
    'wardrobe', 'xbox'
]

In [34]:
modelnet_labels.index("laptop")

20

In [9]:
def parahome_get_pcs(path, num_points=1024):
    #Find objects paths
    parahome_objects = os.listdir(path)
    parahome_objects_paths = list(
        map(
            lambda name: os.path.join(path, name),
            parahome_objects
        )
    )
    
    #load
    point_clouds = []
    for mesh_path in parahome_objects_paths:
        mesh = o3d.io.read_triangle_mesh(mesh_path)
        mesh.compute_vertex_normals()
        pcd = mesh.sample_points_uniformly(number_of_points=num_points)
        point_clouds.append(torch.from_numpy(np.asarray(pcd.points, dtype=np.float32)))
    return torch.stack(point_clouds, dim=0)

In [10]:
parahome_pcs = parahome_get_pcs("/Users/photosartd/repositories/ParaHome/data/objects_combined/", 1024)

In [11]:
def display_mesh(mesh):
    def close_window_callback(vis):
        vis.close()  # This will close the visualizer window
    # Create a Visualizer instance
    vis = vis = o3d.visualization.VisualizerWithKeyCallback()
    vis.create_window()

    # Add point cloud to the visualizer
    vis.add_geometry(mesh)

    # Register the key callback for the 'Q' key to close the window
    vis.register_key_callback(ord('Q'), close_window_callback)

    # Run the visualizer
    vis.run()

    # Destroy the window when done
    vis.destroy_window()

In [131]:
def display_open3d(template, paint_uniform=False):
    
    def close_window_callback(vis):
        vis.close()  # This will close the visualizer window
        
    # Create a Visualizer instance
    vis = vis = o3d.visualization.VisualizerWithKeyCallback()
    vis.create_window()
    
    colors = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    
    if not isinstance(template, list):
        template = [template]
    for i, t in enumerate(template):
        
        template_ = o3d.geometry.PointCloud()
        template_.points = o3d.utility.Vector3dVector(t)
        if paint_uniform:
            template_.paint_uniform_color(colors[i])
        # template_.paint_uniform_color([1, 0, 0])

        # Add point cloud to the visualizer
        vis.add_geometry(template_)
            

    # Register the key callback for the 'Q' key to close the window
    vis.register_key_callback(ord('Q'), close_window_callback)

    # Run the visualizer
    vis.run()

    # Destroy the window when done
    vis.destroy_window()

In [30]:
display_open3d(parahome_pcs[16]) #laptop

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [39]:
batch[0].shape

torch.Size([32, 1024, 3])

In [43]:
for batch in test_loader:
    if 20 in batch[1].flatten():
        break

In [45]:
batch[1]

tensor([[32],
        [21],
        [33],
        [22],
        [10],
        [21],
        [30],
        [22],
        [ 5],
        [15],
        [ 7],
        [31],
        [33],
        [ 0],
        [18],
        [20],
        [32],
        [23],
        [12],
        [21],
        [20],
        [16],
        [33],
        [37],
        [34],
        [36],
        [27],
        [37],
        [30],
        [22],
        [21],
        [37]])

In [46]:
display_open3d(batch[0][15]) #laptop

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Registration

In [63]:
display_open3d([np.asarray(res_parahome.points), batch[0][15]])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [64]:
display_open3d([parahome_pcs[16], batch[0][15]])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [215]:
def calculate_scale(source, target):
    source_size = np.linalg.norm(np.max(np.asarray(source.points), axis=0) - np.min(np.asarray(source.points), axis=0))
    target_size = np.linalg.norm(np.max(np.asarray(target.points), axis=0) - np.min(np.asarray(target.points), axis=0))
    return target_size / source_size

def apply_scale(pcd, scale):
    pcd.scale(scale, center=pcd.get_center())
    return pcd

In [216]:
parahome_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(parahome_pcs[16]))
modelnet_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(batch[0][15]))

scale = calculate_scale(parahome_pcd, modelnet_pcd)
scaled_laptop = apply_scale(parahome_pcd, scale)

In [218]:
display_open3d([np.asarray(scaled_laptop.points), batch[0][15]], True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [165]:
def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)
    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    return pcd_down

def register_point_clouds_non_rigid(source, target, voxel_size):
    source_down = preprocess_point_cloud(source, voxel_size)
    target_down = preprocess_point_cloud(target, voxel_size)

    threshold = voxel_size * 1.5
    trans_init = np.eye(4)

    # Use non-rigid ICP for alignment
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    icp_result = o3d.pipelines.registration.registration_icp(
        source_down, target_down, threshold, trans_init,
        p2p, o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200)
    )

    # Non-rigid registration can be achieved using deformation graphs in Open3D
    return icp_result.transformation

# Example usage
voxel_size = 0.01
pc1 = [scaled_laptop]
pc2 = [modelnet_pcd]

for src, tgt in zip(pc1, pc2):
    transformation = register_point_clouds_non_rigid(src, tgt, voxel_size)
    res_parahome = src.transform(transformation)

In [242]:
def rotation_matrix(axis, angle):
    """
    Create a rotation matrix to rotate around a specific axis by a given angle.
    :param axis: The axis of rotation ('x', 'y', or 'z')
    :param angle: The angle of rotation in radians
    :return: A 3x3 rotation matrix
    """
    if axis == 'x':
        R = np.array([[1, 0, 0],
                      [0, np.cos(angle), -np.sin(angle)],
                      [0, np.sin(angle), np.cos(angle)]])
    elif axis == 'y':
        R = np.array([[np.cos(angle), 0, np.sin(angle)],
                      [0, 1, 0],
                      [-np.sin(angle), 0, np.cos(angle)]])
    elif axis == 'z':
        R = np.array([[np.cos(angle), -np.sin(angle), 0],
                      [np.sin(angle), np.cos(angle), 0],
                      [0, 0, 1]])
    else:
        raise ValueError("Invalid axis. Choose from 'x', 'y', or 'z'.")
    return R

def homogeneous_transformation_matrix(rotation_axis, rotation_angle, translation):
    """
    Create a homogeneous transformation matrix for rotation and translation.
    :param rotation_axis: The axis of rotation ('x', 'y', or 'z')
    :param rotation_angle: The angle of rotation in radians
    :param translation: The translation vector as a tuple or list (tx, ty, tz)
    :return: A 4x4 homogeneous transformation matrix
    """
    R = rotation_matrix(rotation_axis, rotation_angle)
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = translation
    return T

def random_transformations(rotation_axis="x", n=10, t_mult=0.1):
    ts = []
    angle_between = np.deg2rad(360 / n)
    curr_angle = 0
    for _ in range(n):
        translation = np.random.randn(3) * t_mult
        ts.append(homogeneous_transformation_matrix(rotation_axis, curr_angle, translation))
        curr_angle += angle_between
    return ts

In [167]:
display_open3d([np.asarray(res_parahome.points), batch[0][15], np.asarray(scaled_laptop.points)], True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [159]:
np.asarray(res_parahome.points)

array([[-0.27236954,  0.3167724 ,  0.36002285],
       [-0.24773921, -0.12096348,  0.29761687],
       [-0.52658969,  0.3323254 ,  0.44830712],
       ...,
       [ 0.01162417,  0.25794488,  0.2139663 ],
       [ 0.31073272, -0.09721488,  0.16651023],
       [ 0.39070681,  0.67960052,  0.50197664]])

In [160]:
np.asarray(parahome_pcd.points)

array([[-0.27236954,  0.3167724 ,  0.36002285],
       [-0.24773921, -0.12096348,  0.29761687],
       [-0.52658969,  0.3323254 ,  0.44830712],
       ...,
       [ 0.01162417,  0.25794488,  0.2139663 ],
       [ 0.31073272, -0.09721488,  0.16651023],
       [ 0.39070681,  0.67960052,  0.50197664]])

In [161]:
transformation

array([[ 9.99999021e-01,  6.90912759e-04, -1.21663607e-03,
         5.20245744e-04],
       [-6.90598358e-04,  9.99999728e-01,  2.58819951e-04,
         1.14079416e-03],
       [ 1.21681457e-03, -2.57979491e-04,  9.99999226e-01,
        -1.58531826e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

# Registration networks

In [157]:
from learning3d.models import PointNet, PointNetLK, RPMNet, PPFNet, DGCNN, DCP, DeepGMR
from learning3d.losses import FrobeniusNormLoss, RMSEFeaturesLoss
from learning3d.data_utils import RegistrationData, ModelNet40Data

In [266]:
# Create PointNet Model.
ptnet = PointNet(emb_dims=1024, use_bn=True)
model = pnlk = PointNetLK(feature_model=ptnet, delta=1e-02, xtol=1e-07, p0_zero_mean=True, p1_zero_mean=True, pooling='max')

In [267]:
pnlk_path = "/Users/photosartd/repositories/learning3d/pretrained/exp_pnlk/models/best_model.t7"
model.load_state_dict(torch.load(pnlk_path, map_location='cpu'))

<All keys matched successfully>

In [268]:
srcpts = np.asarray(scaled_laptop.transform(random_transformations("x")[0]).points, dtype=np.float32)

In [269]:
with torch.no_grad():
    output = model(batch[0][15].unsqueeze(0), torch.from_numpy(srcpts).unsqueeze(0))

In [270]:
batch[0][15].shape

torch.Size([1024, 3])

In [271]:
parahome_pcs[16].shape

torch.Size([1024, 3])

In [272]:
output['transformed_source'].detach().cpu().numpy()[0].shape

(1024, 3)

In [273]:
output['transformed_source'].detach().cpu().numpy()[0]

array([[-0.4611162 ,  0.2915361 ,  0.30319312],
       [-0.3437891 ,  0.5148101 , -0.06080817],
       [-0.49840236,  0.49994123,  0.4700491 ],
       ...,
       [-0.3676174 ,  0.07155377,  0.08339466],
       [-0.33098426,  0.03822261, -0.38072106],
       [-0.7817253 , -0.39866334,  0.19228753]], dtype=float32)

In [274]:
display_open3d([output['transformed_source'].detach().cpu().numpy()[0], batch[0][15], srcpts], True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [255]:
np.sqrt((output['r'] ** 2).sum())

tensor(11.5693)

In [256]:
best_res = np.inf
best_out = None
best_start = None
for T in random_transformations("x"):
    with torch.no_grad():
        srcpts = np.array(scaled_laptop.transform(T).points, dtype=np.float32)
        srcpts = torch.from_numpy(srcpts).unsqueeze(0)
        output = model(batch[0][15].unsqueeze(0), srcpts)
        error = np.sqrt((output['r'] ** 2).sum())
        if error < best_res:
            best_res = error
            best_out = output['transformed_source'].detach().cpu().numpy()[0]
            best_start = srcpts
display_open3d([best_out, batch[0][15], srcpts[0]], True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [259]:
model = DeepGMR(use_rri=True, nearest_neighbors=20)

In [262]:
model.load_state_dict(torch.load("/Users/photosartd/repositories/learning3d/pretrained/exp_deepgmr/models/best_model.pth", map_location="cpu"), strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['backbone.encoder.0.0.weight', 'backbone.encoder.0.1.weight', 'backbone.encoder.0.1.bias', 'backbone.encoder.0.1.running_mean', 'backbone.encoder.0.1.running_var', 'backbone.encoder.0.1.num_batches_tracked', 'backbone.encoder.1.0.weight', 'backbone.encoder.1.1.weight', 'backbone.encoder.1.1.bias', 'backbone.encoder.1.1.running_mean', 'backbone.encoder.1.1.running_var', 'backbone.encoder.1.1.num_batches_tracked', 'backbone.encoder.2.0.weight', 'backbone.encoder.2.1.weight', 'backbone.encoder.2.1.bias', 'backbone.encoder.2.1.running_mean', 'backbone.encoder.2.1.running_var', 'backbone.encoder.2.1.num_batches_tracked', 'backbone.encoder.3.0.weight', 'backbone.encoder.3.1.weight', 'backbone.encoder.3.1.bias', 'backbone.encoder.3.1.running_mean', 'backbone.encoder.3.1.running_var', 'backbone.encoder.3.1.num_batches_tracked', 'backbone.decoder.0.0.weight', 'backbone.decoder.0.1.weight', 'backbone.decoder.0.1.bias', 'backbone.decoder.0.1.run

In [258]:
best_res = np.inf
best_out = None
best_start = None
for T in random_transformations("x"):
    with torch.no_grad():
        srcpts = np.array(modelnet_pcd.transform(T).points, dtype=np.float32)
        srcpts = torch.from_numpy(srcpts).unsqueeze(0)
        output = model(batch[0][15].unsqueeze(0), srcpts)
        error = np.sqrt((output['r'] ** 2).sum())
        if error < best_res:
            best_res = error
            best_out = output['transformed_source'].detach().cpu().numpy()[0]
            best_start = srcpts
print(error)
display_open3d([best_out, batch[0][15], srcpts[0]], True)

tensor(3.6161e-06)
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [171]:
testset = RegistrationData('PointNetLK', ModelNet40Data(train=True, num_points=1024, download=True))
test_loader = DataLoader(testset, batch_size=8, shuffle=False, drop_last=False)

In [275]:
for i, data in enumerate(tqdm(test_loader)):
    template, source, igt = data

    output = model(template, source)

    
    display_open3d([template.detach().cpu().numpy()[0], source.detach().cpu().numpy()[0], output['transformed_source'].detach().cpu().numpy()[0]],
                  True)
    break

  0%|                                                  | 0/1230 [00:00<?, ?it/s]

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


  0%|                                                  | 0/1230 [00:10<?, ?it/s]


In [154]:
#testset = RegistrationData('DCP', ModelNet40Data(train=True, num_points=1024, use_normals=True))
#test_loader = DataLoader(testset, batch_size=1, shuffle=False, drop_last=False)
dgcnn = DGCNN(emb_dims=512)
model = DCP(feature_model=dgcnn, cycle=True)

In [156]:
model.load_state_dict(torch.load("/Users/photosartd/repositories/learning3d/pretrained/exp_dcp/models/best_model.t7",
                                map_location="cpu"), strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['pointer.model.encoder.layers.0.sublayer.0.norm.norm.weight', 'pointer.model.encoder.layers.0.sublayer.0.norm.norm.bias', 'pointer.model.encoder.layers.0.sublayer.0.norm.norm.running_mean', 'pointer.model.encoder.layers.0.sublayer.0.norm.norm.running_var', 'pointer.model.encoder.layers.0.sublayer.0.norm.norm.num_batches_tracked', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.weight', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.bias', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.running_mean', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.running_var', 'pointer.model.encoder.layers.0.sublayer.1.norm.norm.num_batches_tracked', 'pointer.model.encoder.norm.norm.weight', 'pointer.model.encoder.norm.norm.bias', 'pointer.model.encoder.norm.norm.running_mean', 'pointer.model.encoder.norm.norm.running_var', 'pointer.model.encoder.norm.norm.num_batches_tracked', 'pointer.model.decoder.layers.0.sublayer.0.norm.norm

In [ ]:
next(iter(test_loader))[0].shape